## numpy实现一个完整神经网络

基本上参照d2l_08.ipynb，更彻底的用numpy来实现一下

In [897]:
import numpy as np

In [898]:
# 先把工具函数定义出来

# 输出函数
def softmax(x):
    c = np.max(x)
    exp_a = np.exp(x - c) # 减去c是为了防止溢出
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

# 激活函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 损失函数
def cross_entropy_error(y, t):
    # 由于loge(0)是负无穷大-inf，计算机无法继续之后的运算
    # 所以给输入增加一个微小的数，并且不影响结果
    delta = 1e-7
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + delta)) / batch_size


In [929]:
# 输入是X(1,5), W1是(5, 8), W2是（8， 3） 然后softmax输出 一个简单版的MINIST
# b偷懒用一个数了

true_w1 = np.random.normal(0.0, 8.0, (5, 8))
true_b1 = 2.15
true_w2 = np.random.normal(0.0, 5.0, (8, 3))
true_b2 = -1.32


# 准备模拟数据 重复生成数据中，找到分布比较均的
def synthetic_data(num):
    
    X = np.random.normal(0.0, 3.0, (num, 5))
    y1 = X.dot(true_w1) + true_b1
    y1 = sigmoid(y1)
    y2 = y1.dot(true_w2) + true_b2
    # 加上个随机扰动
    y2 += np.random.random(y2.shape)
    y2 = softmax(y2)
    
    # 把概率最高的设置为1
    y = np.zeros((num, 3))
    row_indices = y2.argmax(axis = 1)
    column_indices = np.arange(num)
    y[column_indices, row_indices] = 1
    
    #统计下y
    elements, counts = np.unique(row_indices, return_counts=True)
    for e, c in zip(elements, counts):
        print(f'{e}出现了{c}次')    
    return X, y

# 设置np的打印选项：精度最大为小数点后2位，不用科学计数法
np.set_printoptions(precision=4, suppress=True)

features, labels = synthetic_data(10000)


0出现了7731次
1出现了1524次
2出现了745次


In [930]:
# 批量样本生成器
def data_iter(batch_size, features, labels):
    #随机索引
    num_examples = features.shape[0]
    indices = np.arange(num_examples)
    indices = np.random.permutation(indices)
    for i in range(0, num_examples, batch_size):
        start = i
        end = min(i + batch_size, num_examples)
        select_indices = indices[start:end]
        yield features[select_indices], labels[select_indices]

In [932]:
# 网络
def net(X, paramas):
    w1 = paramas[w1]
    b1 = paramas[b1]
    w2 = paramas[w2]
    b2 = paramas[b2]
    
    y1 = X.dot(w1) + b1
    y1 = sigmoid(y1)
    y2 = y1.dot(w2) + b2
    y = softmax(y2)
    return y

# 损失函数
def loss(x, t, paramas):
    y = net(x, paramas)
    return cross_entropy_error(y, t)

